# 1. Information about the submission

## 1.1 Name and number of the assignment

Task 2. Detoxification through ruPROMPTs and metric hacking

## 1.2 Student name

Yurchenko Vladislav

## 1.3 Codalab user ID

v-yurchenko

# 2. Technical Report

## 2.1 Methodology 

Использовались 2 подхода: 
 - P-tuning LM модели с чекпоинта sberbank-ai/rugpt3large_based_on_gpt2 
 - Zero-shot learning модели OpenAI через открытое API
 
В первом подходе мы дообучали модель LM на примерах из датасета.
На inference производилась генерация 10 примеров, для каждого примера считались метрики и Joint Score.
Результаты сортировались по максимальному Joint Score. 
Выбирался ответ с максимальным скором.

Во втором подходе использовался zero-shot подход для ЯМ chatGPT. Предлагалось переформулировать текст через промпт.
**Переформулируй текст без вульгарных выражений:**

Выбирался первый ответ chatGPT.

## 2.2 Discussion of results

Мои эксперименты:

| Checkpoint | Additional info | Joint score | Leaderbord |
| :- | :- | -: | -: |
| baseline |  | - | 0.36  |
| baseline (prev t5) |  | - | 0.45  |
| ruPrompts | - | 0.7274969536 | 0.66 |
| openAI zeroPrompt | - | 0.3600979288 | - |

Первый подход я бы назвал "metric hacking". Мы дообучаем модель переформулировать текст. А далее улучшаем результаты генерации модели за счет того, что отбираем из Топ-10 результатов ответы с наилучшим скором.

Второй подход показал себя хуже, так как без указания примеров модель ChatGPT начинала генерировать ответы достаточно в пространной манере, чем ухудшала метрики по стилю. Текст сгенерированный моделью был далек от исходного.

Идеи по улучшению модели:
 - попробовать few-shot learning подход для модели chatGPT - формировать промпт, который укажет модели как стиль в котором лучше генерировать ответы
 - использовать метрику Joint Score для доранжирования результатов, 
 - если ранжировщик Joint Score будет медленный для наших целей (а он такой и есть), то обучить дополнительный ранжировщик ответов 


# 3. Code

### 3.1 Requirements

In [ ]:
%pip install ruprompts datasets

In [2]:
!mkdir task2/

mkdir: cannot create directory ‘task2/’: File exists


In [3]:
!wget -O task2/train.tsv https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
!wget -O task2/dev.tsv https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
!wget -O task2/test.tsv https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/test.tsv

--2023-04-23 00:12:42--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/train.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1902888 (1.8M) [text/plain]
Saving to: ‘task2/train.tsv’

task2/train.tsv     100%[===================>]   1.81M  8.35MB/s    in 0.2s    

2023-04-23 00:12:43 (8.35 MB/s) - ‘task2/train.tsv’ saved [1902888/1902888]

--2023-04-23 00:12:43--  https://raw.githubusercontent.com/skoltech-nlp/russe_detox_2022/main/data/input/dev.tsv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Leng

In [16]:
!wget https://raw.githubusercontent.com/v-yurchenko/transformers/main/ru_detoxification_evaluation.py
!wget https://raw.githubusercontent.com/v-yurchenko/transformers/main/ru_detoxification_metrics.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
--2023-04-23 16:29:38--  https://raw.githubusercontent.com/v-yurchenko/transformers/main/ru_detoxification_evaluation.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3606 (3.5K) [text/plain]
Saving to: ‘ru_detoxification_evaluation.py’

ru_detoxification_e 100%[===================>]   3.52K  --.-KB/s    in 0s      

2023-04-23 16:29:38 (47.9 MB/s) - ‘ru_detoxification_evaluation.py’ saved [3606/3606]

huggingface/tokenizers: The current 

### 3.2 P-tuning

In [4]:
import pandas as pd

df = pd.read_csv("task2/train.tsv", sep="\t")
df.drop(["index"], axis=1, inplace=True, errors='ignore')
df.to_csv("task2/train.tsv", index=False, sep="\t")

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [6]:
from datasets import load_dataset

datasets = load_dataset("csv", data_files={
    "train": "task2/train.tsv", 
    "validation": "task2/dev.tsv",
}, sep="\t")
train_dataset = datasets["train"]
valid_dataset = datasets["validation"]

Found cached dataset csv (/home/jovyan/.cache/huggingface/datasets/csv/default-9360cce579f368c3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

In [7]:
from transformers import GPT2LMHeadModel, AutoTokenizer

backbone_id = "sberbank-ai/rugpt3large_based_on_gpt2"

model = GPT2LMHeadModel.from_pretrained(backbone_id)
tokenizer = AutoTokenizer.from_pretrained(backbone_id, pad_token="<pad>", eos_token="<pad>")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
from ruprompts import PromptFormat

prompt_format = PromptFormat("<P*100>{toxic_comment}<P*20>")

In [9]:
from ruprompts import TensorPromptProvider
from transformers import set_seed

set_seed(1)

prompt_provider = TensorPromptProvider()

In [10]:
from ruprompts import Prompt

prompt = Prompt(prompt_format, prompt_provider)
prompt.patch(model, tokenizer)

In [11]:
from ruprompts import Text2TextPreprocessor

preprocessor = Text2TextPreprocessor(
    prompt_format=prompt_format,
    tokenizer=tokenizer,
    target_field="neutral_comment1",
    max_tokens=1792,
    truncation_field="toxic_comment",
)

train_dataset = train_dataset.map(preprocessor)
valid_dataset = valid_dataset.map(preprocessor)

Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-9360cce579f368c3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-44d93aff7c544c41.arrow
Loading cached processed dataset at /home/jovyan/.cache/huggingface/datasets/csv/default-9360cce579f368c3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-af3df78b33fd78f1.arrow


In [12]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir=".",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=1,
    eval_steps=1000,
    save_steps=1000,
    logging_steps=100,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_strategy="steps",
    save_total_limit=2,
    metric_for_best_model="eval_loss",
    learning_rate=0.1,
    max_steps=100000,
    report_to="tensorboard",
    # report_to=["tensorboard", "wandb"],  # uncomment to log to WandB
    logging_dir="logs",
    seed=1,
)

In [13]:
from transformers.optimization import AdamW, get_linear_schedule_with_warmup

optimizer = AdamW(prompt_provider.parameters(), lr=training_args.learning_rate)
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=2000,
    num_training_steps=training_args.max_steps,
)

/home/user/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


In [ ]:
from transformers import Trainer
from ruprompts.callbacks import (
    FreezeTransformerUnfreezePrompt,
    ReduceCheckpoint,
    SavePretrainedPrompt,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    data_collator=preprocessor.collate_fn(),
    optimizers=(optimizer, scheduler),
    callbacks=[FreezeTransformerUnfreezePrompt(), ReduceCheckpoint(), SavePretrainedPrompt(prompt)],
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
1000,2.201700,1.911890
2000,1.692000,1.783948
3000,1.598500,1.709484
4000,1.633900,1.644775
5000,1.541400,1.598003
6000,1.516300,1.548342
7000,1.483300,1.600356
8000,1.574000,1.540647
9000,1.488500,1.475800
10000,1.447200,1.489802


### 3.3 Inference

In [14]:
from transformers import pipeline

prompt = Prompt.from_pretrained(f"./checkpoint-{training_args.max_steps}")

ppln = pipeline("text2text-generation-with-prompt", prompt=prompt, model=model, tokenizer=tokenizer, device=0)

In [15]:
query = "Ублюдок, мать твою, а ну иди сюда"
options = ppln({"toxic_comment": query}, 
            do_sample=False,           
            num_beams=10,
            num_return_sequences=10
              )
options

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


[{'generated_text': 'А ну иди сюда'},
 {'generated_text': 'а ну иди сюда'},
 {'generated_text': 'Ну иди сюда'},
 {'generated_text': 'Плохой человек, а ну иди сюда'},
 {'generated_text': 'Ну, иди сюда'},
 {'generated_text': 'Уходи'},
 {'generated_text': 'Отстань от меня'},
 {'generated_text': 'Отстань'},
 {'generated_text': 'А ну давай сюда'},
 {'generated_text': 'А ну, иди сюда'}]

### 3.4 Metric hacking

In [17]:
# используем метрики для соревнования и попробуем посчитать скор для нашей пары - запрос / ответ
use_cuda = True

from ru_detoxification_evaluation import load_model
from ru_detoxification_metrics import evaluate_style

from transformers import AutoTokenizer, AutoModelForSequenceClassification, AutoModel
from ru_detoxification_metrics import evaluate_cosine_similarity
from ru_detoxification_metrics import evaluate_cola_relative

style_model, style_tokenizer = load_model('SkolkovoInstitute/russian_toxicity_classifier', use_cuda=use_cuda)

accuracy = evaluate_style(
    model = style_model,
    tokenizer = style_tokenizer,
    texts = [query],
    target_label=0,  # 1 is toxic, 0 is neutral
    batch_size=32, 
    verbose=True
)
accuracy

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.10729373], dtype=float32)

In [18]:
meaning_model, meaning_tokenizer = load_model('cointegrated/LaBSE-en-ru', use_cuda=use_cuda, model_class=AutoModel)
similarity = evaluate_cosine_similarity(
    model = meaning_model,
    tokenizer = meaning_tokenizer,
    original_texts = [query],
    rewritten_texts = [options[0]['generated_text']],
    batch_size=32,
    verbose=True,
    )
similarity

Some weights of the model checkpoint at cointegrated/LaBSE-en-ru were not used when initializing BertModel: ['cls.seq_relationship.weight', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.42332578], dtype=float32)

In [19]:
cola_model, cola_tolenizer = load_model('SkolkovoInstitute/rubert-base-corruption-detector', use_cuda=use_cuda)
fluency = evaluate_cola_relative(
    model = cola_model,
    tokenizer = cola_tolenizer,
    original_texts = [query],
    rewritten_texts = [options[0]['generated_text']],
    target_label=1,
    batch_size=32,
    verbose=True
)
fluency

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.9671553], dtype=float32)

In [20]:
joint = accuracy * similarity * fluency
joint

array([0.04392838], dtype=float32)

In [22]:
from nltk.translate.chrf_score import corpus_chrf

def calc_score(toxic_text, prediction):
    accuracy = evaluate_style(
    model = style_model,
    tokenizer = style_tokenizer,
    texts = [prediction],
    target_label=0,  # 1 is toxic, 0 is neutral
    batch_size=32, 
    verbose=True
    )
    
    similarity = evaluate_cosine_similarity(
    model = meaning_model,
    tokenizer = meaning_tokenizer,
    original_texts = [toxic_text],
    rewritten_texts = [prediction],
    batch_size=32,
    verbose=True,
    )
    
    fluency = evaluate_cola_relative(
    model = cola_model,
    tokenizer = cola_tolenizer,
    original_texts = [toxic_text],
    rewritten_texts = [prediction],
    target_label=1,
    batch_size=32,
    verbose=True
    )
    
    return accuracy * similarity * fluency
    
calc_score(query, options[3]['generated_text'])

# а это хитрое расстояние от нейтрального текста (в тестовой выборке) к переписанному, как его тут учесть?
# corpus_chrf(neutral_references, rewritten)

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

array([0.07616657], dtype=float32)

### 3.5. RUN INFERENCE

In [ ]:
from tqdm import tqdm
import transformers
import numpy as np

# будем скорить таким образом:
#  - генерируем 10 кандидатов с помощью модели
#  - скорим их по нашей метрике Joint Score
#  - выбираем кандидата с максимальным скором
transformers.logging.set_verbosity_error()

beam_count = 10 

predictions = []

with open("task2/test.tsv", "r") as test_f:
    with open("task2/test_submission.txt", "w") as f:
        for line in test_f:
            query = line.strip("\r\n")
            options = ppln(
                {"toxic_comment": query},
                do_sample=False,
                num_beams=beam_count,
                num_return_sequences=beam_count,
            )

            options = [opt["generated_text"].replace("<pad>", "") for opt in options]
            scores  = [calc_score(query, opt) for opt in options]
            answer = options[np.argmax(scores)]
            predictions.append(answer)
            f.write(answer.replace("\n", "") + "\n")

### 3.6. OpenAI interface

In [ ]:
!pip install openai

In [ ]:
# ключ к API не дам, он у меня лимитированный на другую команду :))
import os
import openai
openai.api_key = "XXX"

In [ ]:
def rewrite_open_ai(text):
    query = f"Переформулируй текст без вульгарных выражений: {text}"
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
        messages=[
        {"role": "user", "content": query}
      ] ,
      # prompt=query,
      temperature=0,
      max_tokens=500,
      top_p=1.0,
      frequency_penalty=0.0,
      presence_penalty=0.0,
      stop=["\n"]
    )
    return response['choices'][0]['message']['content']

rewrite_open_ai("какой дебил эту придумывает ")

In [ ]:
import time
time.sleep(0.5)

In [ ]:
with open("task2/test.tsv", "r") as test_f:
    with open("task2/test_open_ai_submission.txt", "w") as f:
        for idx, line in enumerate(test_f):
            # ядро периодически почему-то падает, поэтому приходилось перезапускать скоринг 
            # и выполнять с какой-то строчки
            if idx < 718:
                continue
            answer = rewrite_open_ai(line.strip("\r\n"))
            time.sleep(0.5)
            print(idx, '-> ', line.strip("\r\n"), ' -> ', answer)
            f.write(answer.replace("\n", "") + "\n")